# transfer learning with keras (tenorflow backend)

In [ ]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import model_from_yaml
from keras import models, layers, optimizers
from keras.applications import VGG16

# we will be using only the convolution part of the model, so we will keep include_top = False, for using the whole model as such use include_top = True. input shape will be the dimensions of the image on the model was trained. weights will be of imagenet.

In [ ]:
conv_base = VGG16(weights = 'imagenet', include_top = False, input_shape = (224, 224, 3))

#provide the path for training and validation path. the directory should contain other sub-directories #belonging to their specific classes.

In [ ]:
train_dir = r'path_do_training_directory'
validation_dir = r'path_do_validation_directory'

batch_size = 20
n_classes = 'no_of_classes_to_train_for'
n_train = 'no_of_training_images'
n_valid = 'no_of_validation_images'

# create a data generator to flow the data from directory to the conv_base for both training and validation data.

In [ ]:
datagen = ImageDataGenerator(rescale = 1./255)
train_features = np.zeros(shape = (n_train, 7, 7, 512))
train_labels = np.zeros(shape = (n_train, n_classes))

train_generator = datagen.flow_from_directory(train_dir,
                                             target_size = (224, 224),
                                             batch_size = batch_size,
                                             class_mode = 'categorical',
                                             shuffle = False)

In [ ]:
i = 0
for inputs_batch, labels_batch in train_generator:
    features_batch = conv_base.predict(inputs_batch)
    train_features[i * batch_size : (i + 1) * batch_size] = features_batch
    train_labels[i * batch_size : (i + 1) * batch_size] = labels_batch
    i += 1
    if i * batch_size >= n_train:
        break
         
train_features = np.reshape(train_features, (n_train, 7 * 7 * 512))

# creating our own fully connected neural network to do the classification, using keras.

In [ ]:
model = models.Sequential()

model.add(layers.Dense(units = 256, activation = 'relu', input_dim = 7*7*512))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(2, activation = 'softmax'))

# training our fully connected network

In [ ]:
model.compile(optimizer=optimizers.RMSprop(lr=2e-4),
              loss='categorical_crossentropy',
              metrics=['acc'])
 
history = model.fit(train_features,
                    train_labels,
                    epochs = 20,
                    batch_size = batch_size,
                    validation_data = (validation_features, validation_labels))

# save the trained model

In [ ]:
# serialize model to YAML
model_yaml = model.to_yaml()
with open("model.yaml", "w") as yaml_file:
    yaml_file.write(model_yaml)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk") 

# for loading the model to memory.

In [ ]:
yaml_file = open('model.yaml', 'r')
loaded_model_yaml = yaml_file.read()
yaml_file.close()
loaded_model = model_from_yaml(loaded_model_yaml)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")
# use loaded_model to compile,fit etc..

# to check the misclassified images

In [ ]:
fnames = train_generator_valid.filenames
 
ground_truth = train_generator_valid.classes
 
label2index = train_generator_valid.class_indices
 
# Getting the mapping from class index to class label
idx2label = dict((v,k) for k,v in label2index.items())
 
predictions = model.predict_classes(validation_features)
prob = model.predict(validation_features)
 
errors = np.where(predictions != ground_truth)[0]
print("No of errors = {}/{}".format(len(errors),n_valid))

In [ ]:
for i in range(len(errors)):
    pred_class = np.argmax(prob[errors[i]])
    pred_label = idx2label[pred_class]
     
    print('Original label:{}, Prediction :{}, confidence : {:.3f}'.format(
        fnames[errors[i]].split('/')[0],
        pred_label,
        prob[errors[i]][pred_class]))
     
    original = Image.open('{}/{}'.format(validation_dir,fnames[errors[i]]))
    plt.imshow(original)
    plt.show()